In [1]:
import numpy as np
import pandas as pd

In [2]:
import mlflow
import mlflow.sklearn



c:\Users\fb5cd\anaconda3\envs\data_analyst\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
mlflow.set_experiment("Sentiment_TFIDF_Word2Vec")


2026/02/08 03:33:36 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.schemas
2026/02/08 03:33:36 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.tables
2026/02/08 03:33:36 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.types
2026/02/08 03:33:36 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.constraints
2026/02/08 03:33:36 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.defaults
2026/02/08 03:33:36 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.comments
2026/02/08 03:33:37 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/02/08 03:33:37 INFO alembic.runtime.migration: Will assume non-transactional DDL.


<Experiment: artifact_location='file:c:/Users/fb5cd/OneDrive/Desktop/irs/sentiment_analysis_project/mlruns/2', creation_time=1770499928083, experiment_id='2', last_update_time=1770499928083, lifecycle_stage='active', name='Sentiment_TFIDF_Word2Vec', tags={}>

In [4]:
df=pd.read_csv(r"D:\Downloads\reviews_badminton\data.csv")
df

,Reviewer Name,Review Title,Place of Review,Up Votes,Down Votes,Month,Review text,Ratings
0,Kamal Suresh,Nice product,"Certified Buyer, Chirakkal",889.0,64.0,Feb 2021,"Nice product, good quality, but price is now r...",4
1,Flipkart Customer,Don't waste your money,"Certified Buyer, Hyderabad",109.0,6.0,Feb 2021,They didn't supplied Yonex Mavis 350. Outside ...,1
2,A. S. Raja Srinivasan,Did not meet expectations,"Certified Buyer, Dharmapuri",42.0,3.0,Apr 2021,Worst product. Damaged shuttlecocks packed in ...,1
3,Suresh Narayanasamy,Fair,"Certified Buyer, Chennai",25.0,1.0,NaN,"Quite O. K. , but nowadays the quality of the...",3
4,ASHIK P A,Over priced,NaN,147.0,24.0,Apr 2016,Over pricedJust â?¹620 ..from retailer.I didn'...,1
...,...,...,...,...,...,...,...,...
8513,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
8514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
8515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
8516,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [5]:
df=pd.read_csv(r"D:\Downloads\reviews_badminton\data.csv")
df

,Reviewer Name,Review Title,Place of Review,Up Votes,Down Votes,Month,Review text,Ratings
0,Kamal Suresh,Nice product,"Certified Buyer, Chirakkal",889.0,64.0,Feb 2021,"Nice product, good quality, but price is now r...",4
1,Flipkart Customer,Don't waste your money,"Certified Buyer, Hyderabad",109.0,6.0,Feb 2021,They didn't supplied Yonex Mavis 350. Outside ...,1
2,A. S. Raja Srinivasan,Did not meet expectations,"Certified Buyer, Dharmapuri",42.0,3.0,Apr 2021,Worst product. Damaged shuttlecocks packed in ...,1
3,Suresh Narayanasamy,Fair,"Certified Buyer, Chennai",25.0,1.0,NaN,"Quite O. K. , but nowadays the quality of the...",3
4,ASHIK P A,Over priced,NaN,147.0,24.0,Apr 2016,Over pricedJust â?¹620 ..from retailer.I didn'...,1
...,...,...,...,...,...,...,...,...
8513,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
8514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
8515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
8516,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [6]:
import re
import nltk
import unicodedata
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = str(text)
    text = unicodedata.normalize("NFKD", text).encode("ascii", "ignore").decode("utf-8")
    text = text.lower()
    text = re.sub(r'pricedjust', 'priced just', text)
    text = re.sub(r'pricejust', 'price just', text)
    text = text.replace("o. k.", "ok").replace("o.k.", "ok")
    text = re.sub(r'read more', '', text)
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    words = [w for w in text.split() if w not in stop_words]

    return " ".join(words)

df['clean_text'] = df['Review text'].apply(clean_text)
df[['Review text', 'clean_text']].head()




[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fb5cd\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Review text,clean_text
0,"Nice product, good quality, but price is now r...",nice product good quality price rising bad sig...
1,They didn't supplied Yonex Mavis 350. Outside ...,supplied yonex mavis outside cover yonex ad in...
2,Worst product. Damaged shuttlecocks packed in ...,worst product damaged shuttlecocks packed new ...
3,"Quite O. K. , but nowadays the quality of the...",quite ok nowadays quality corks like years bac...
4,Over pricedJust â?¹620 ..from retailer.I didn'...,priced retailer understand wat advantage buyin...


In [7]:
import nltk
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

df['normalized_text'] = df['clean_text'].apply(lemmatize_text)

df[['clean_text', 'normalized_text']].head()


pd.set_option('display.max_colwidth', None)

df[['Review text','clean_text', 'normalized_text']].head()


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\fb5cd\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\fb5cd\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,Review text,clean_text,normalized_text
0,"Nice product, good quality, but price is now rising which is a bad sign. 800-850 was an affordable price, especially when we play everyday. So kindly help us out in terms of the price. Thank You.READ MORE",nice product good quality price rising bad sign affordable price especially play everyday kindly help us terms price thank,nice product good quality price rising bad sign affordable price especially play everyday kindly help u term price thank
1,They didn't supplied Yonex Mavis 350. Outside cover was Yonex Ad inside was a cheapest.... Sad to hear this.READ MORE,supplied yonex mavis outside cover yonex ad inside cheapest sad hear,supplied yonex mavis outside cover yonex ad inside cheapest sad hear
2,Worst product. Damaged shuttlecocks packed in new box. It's not a original yonex product. Don't buy.flipkart platform is chosen to fraud the buyers.READ MORE,worst product damaged shuttlecocks packed new box original yonex product buy flipkart platform chosen fraud buyers,worst product damaged shuttlecock packed new box original yonex product buy flipkart platform chosen fraud buyer
3,"Quite O. K. , but nowadays the quality of the corks like not as before 3 to 5 years back.. I am using MAVIS 350 for more than 15 years quality of corks was very very good at that times, but now I am not getting the quality corks as like before, rate of corks also too much now, I am very sorry to say like this, but in my experience , my Statment is very true to my knowledgeREAD MORE",quite ok nowadays quality corks like years back using mavis years quality corks good times getting quality corks like rate corks also much sorry say like experience statment true knowledge,quite ok nowadays quality cork like year back using mavis year quality cork good time getting quality cork like rate cork also much sorry say like experience statment true knowledge
4,Over pricedJust â?¹620 ..from retailer.I didn't understand.. Wat is d advantage of buying dis frm flipkrtREAD MORE,priced retailer understand wat advantage buying dis frm flipkrt,priced retailer understand wat advantage buying dis frm flipkrt


In [8]:
def sentiment_label(rating):
    if rating >= 4:
        return 1
    elif rating <= 2:
        return 0
    else:
        return None

df['sentiment'] = df['Ratings'].apply(sentiment_label)


df = df.dropna(subset=['sentiment'])

df['sentiment'].value_counts()


sentiment
1.0    6826
0.0    1077
Name: count, dtype: int64

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf = TfidfVectorizer(
    max_features=5000,      
    ngram_range=(1,2)       
)

X_tfidf = tfidf.fit_transform(df['normalized_text'])


y = df['sentiment']

print("TF-IDF shape:", X_tfidf.shape)


TF-IDF shape: (7903, 5000)


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print(X_train.shape, X_test.shape)


(6322, 5000) (1581, 5000)


In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

with mlflow.start_run(run_name="LogReg_TFIDF"):

    # ---- PARAMETERS ----
    max_iter = 1000

    # Log parameters
    mlflow.log_param("model", "LogisticRegression")
    mlflow.log_param("max_iter", max_iter)
    mlflow.log_param("vectorizer", "TFIDF")

    # ---- MODEL TRAINING ----
    lr = LogisticRegression(max_iter=max_iter)
    lr.fit(X_train, y_train)

    # ---- TRAIN METRICS ----
    y_train_pred_lr = lr.predict(X_train)
    f1_train_lr = f1_score(y_train, y_train_pred_lr)

    # ---- TEST METRICS ----
    y_test_pred_lr = lr.predict(X_test)
    f1_test_lr = f1_score(y_test, y_test_pred_lr)

    # Log metrics
    mlflow.log_metric("f1_train", f1_train_lr)
    mlflow.log_metric("f1_test", f1_test_lr)

    # ---- LOG MODEL ----
    mlflow.sklearn.log_model(lr, artifact_path="model")

    print("Logistic Regression")
    print("Train F1-score:", f1_train_lr)
    print("Test  F1-score:", f1_test_lr)

2026/02/08 03:33:43 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
c:\Users\fb5cd\anaconda3\envs\data_analyst\Lib\site-packages\mlflow\models\model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


Logistic Regression
Train F1-score: 0.9631081803600071
Test  F1-score: 0.9540960451977402


In [21]:
from sklearn.svm import LinearSVC

with mlflow.start_run(run_name="SVM_TFIDF"):

    mlflow.log_param("model", "LinearSVC")
    mlflow.log_param("vectorizer", "TFIDF")

    svm = LinearSVC()
    svm.fit(X_train, y_train)

    y_train_pred = svm.predict(X_train)
    y_test_pred = svm.predict(X_test)

    f1_train = f1_score(y_train, y_train_pred)
    f1_test = f1_score(y_test, y_test_pred)

    mlflow.log_metric("f1_train", f1_train)
    mlflow.log_metric("f1_test", f1_test)

    mlflow.sklearn.log_model(svm, "model")

    print("TFIDF + SVM")
    print("Train F1:", f1_train)
    print("Test F1:", f1_test)

2026/02/08 03:37:31 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
c:\Users\fb5cd\anaconda3\envs\data_analyst\Lib\site-packages\mlflow\models\model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


TFIDF + SVM
Train F1: 0.9504845814977974
Test F1: 0.9472573839662447


In [23]:
from gensim.models import Word2Vec


tokenized_text = df['normalized_text'].apply(lambda x: x.split())

tokenized_text.head()


0    [nice, product, good, quality, price, rising, bad, sign, affordable, price, especially, play, everyday, kindly, help, u, term, price, thank]
1                                                                [supplied, yonex, mavis, outside, cover, yonex, ad, inside, cheapest, sad, hear]
2               [worst, product, damaged, shuttlecock, packed, new, box, original, yonex, product, buy, flipkart, platform, chosen, fraud, buyer]
4                                                                       [priced, retailer, understand, wat, advantage, buying, dis, frm, flipkrt]
5                                                                                                       [good, quality, product, delivered, time]
Name: normalized_text, dtype: object

In [24]:
w2v_model = Word2Vec(
    sentences=tokenized_text,
    vector_size=100,  
    window=5,
    min_count=2,
    workers=4,
    epochs=20
)


In [25]:
w2v_model.wv['quality']

array([-0.08118875,  0.34638023,  0.06816185, -0.01432574,  0.07135901,
       -0.4744857 ,  0.06140062,  0.77200997, -0.09086091, -0.3487903 ,
        0.06813535, -0.3358295 ,  0.15187126,  0.15855749,  0.19336705,
       -0.147509  ,  0.34018   , -0.2141464 , -0.10948797, -0.66482854,
        0.14621098, -0.12301642,  0.19253935,  0.04259176, -0.46909383,
        0.01168347, -0.16558696, -0.08358705, -0.19031554,  0.12512489,
        0.21573877,  0.17100331,  0.10081412, -0.22547397, -0.03240985,
        0.4084735 ,  0.14393674, -0.47967744, -0.27603927, -0.2496223 ,
        0.12257745, -0.26195848, -0.0117027 , -0.09576116,  0.41920787,
       -0.0246703 ,  0.02094504, -0.38501945,  0.02829049,  0.02686741,
        0.2679018 , -0.04874488, -0.07270397, -0.11991429,  0.07874806,
       -0.01540397,  0.02023229,  0.12213755, -0.13606012, -0.04033253,
        0.03201573, -0.04983403,  0.03998828, -0.1647985 , -0.20759499,
        0.23130482,  0.15621036,  0.14934295, -0.27577087,  0.33

In [26]:
import numpy as np

def sentence_vector(sentence, model, vector_size=100):
    words = sentence.split()
    word_vectors = [
        model.wv[word] for word in words if word in model.wv
    ]
    if len(word_vectors) == 0:
        return np.zeros(vector_size)
    return np.mean(word_vectors, axis=0)


In [27]:
X_w2v = np.array([
    sentence_vector(text, w2v_model, 100)
    for text in df['normalized_text']
])

y = df['sentiment']

print(X_w2v.shape)


(7903, 100)


In [28]:
y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(
    X_w2v, y, test_size=0.2, random_state=42, stratify=y
)

In [33]:
with mlflow.start_run(run_name="LogReg_W2V"):

    # ---- PARAMETERS ----
    max_iter = 1000
    mlflow.log_param("model", "LogisticRegression")
    mlflow.log_param("vectorizer", "Word2Vec")
    mlflow.log_param("max_iter", max_iter)

    # ---- MODEL TRAINING ----
    lr = LogisticRegression(max_iter=max_iter)
    lr.fit(X_train, y_train)

    # ---- PREDICTIONS ----
    y_train_pred = lr.predict(X_train)
    y_test_pred  = lr.predict(X_test)

    # ---- METRICS ----
    f1_train = f1_score(y_train, y_train_pred)
    f1_test  = f1_score(y_test, y_test_pred)

    # Log metrics
    mlflow.log_metric("f1_train", f1_train)
    mlflow.log_metric("f1_test", f1_test)

    # ---- LOG MODEL ----
    mlflow.sklearn.log_model(lr, name="model")

    # ---- PRINT RESULTS ----
    print("Model       : Logistic Regression")
    print("Vectorizer  : Word2Vec")
    print(f"Train F1    : {f1_train:.4f}")
    print(f"Test  F1    : {f1_test:.4f}")


c:\Users\fb5cd\anaconda3\envs\data_analyst\Lib\site-packages\mlflow\models\model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


Model       : Logistic Regression
Vectorizer  : Word2Vec
Train F1    : 0.9482
Test  F1    : 0.9454


In [31]:
svm = LinearSVC()

with mlflow.start_run(run_name="SVM_W2V"):

    # ---- PARAMETERS ----
    mlflow.log_param("model", "LinearSVC")
    mlflow.log_param("vectorizer", "Word2Vec")

    # ---- MODEL TRAINING ----
    svm = LinearSVC()
    svm.fit(X_train, y_train)

    # ---- PREDICTIONS ----
    y_train_pred = svm.predict(X_train)
    y_test_pred  = svm.predict(X_test)

    # ---- METRICS ----
    f1_train = f1_score(y_train, y_train_pred)
    f1_test  = f1_score(y_test, y_test_pred)

    # Log metrics to MLflow
    mlflow.log_metric("f1_train", f1_train)
    mlflow.log_metric("f1_test", f1_test)

    # ---- LOG MODEL ----
    mlflow.sklearn.log_model(svm, name="model")

    # ---- PRINT RESULTS ----
    print("Model       : LinearSVC")
    print("Vectorizer  : Word2Vec")
    print(f"Train F1    : {f1_train:.4f}")
    print(f"Test  F1    : {f1_test:.4f}")

c:\Users\fb5cd\anaconda3\envs\data_analyst\Lib\site-packages\mlflow\models\model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


Model       : LinearSVC
Vectorizer  : Word2Vec
Train F1    : 0.9504
Test  F1    : 0.9473
